In [1]:
#!/usr/bin/python3 -m pip install spacy
#!/usr/bin/python3 -m spacy download en_core_web_sm 


2021-03-14 21:16:38.951053: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-03-14 21:16:38.951106: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import numpy as np
from numpy import linspace, loadtxt, ones, convolve
import pandas as pd
#import pickle
import json
import os
import nltk
from nltk import RegexpTokenizer
#from collections import Counter
import re
from datetime import datetime
import time
#from scipy.signal import savgol_filter
import string
import spacy
nlp = spacy.load("en_core_web_sm")

from gensim import matutils, models
import scipy.sparse

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
global_path = '~/Desktop/Datasets/art/art_writing'
path = 'artnet_articles'
#path = 'artnet_articles'
#path = 'artnet_articles'
folderpath = os.path.expanduser(os.path.join(global_path, path))

filelist = os.listdir(folderpath)

In [4]:
data = {}
data = pd.DataFrame(data)

cleaned_titles = []
cleaned_paras = []
brought_in = 0

for file in filelist: #[:100]:
    #file = filelist.sample(1000)
    filepath = os.path.join(folderpath, file)
    f = open(filepath) #, encoding='ascii', errors='ignore')
    try:
        j_import = json.load(f)
        j_import = j_import[0]
        #print(j_import)
        #title = j_import['title']
        try:
            para = j_import['para']
            para = para[0]
            tokenizer = RegexpTokenizer(r'\w+')
            tokens = tokenizer.tokenize(para)
            #print(tokens)
            count = len(tokens)
            if count > 1000 and brought_in < 100:
                tokens =[]
                brought_in += 1
                
                para = para.replace("	", "").replace("Follow  on Facebook:", '').replace("\n", ' ')
                para = para.replace("\r" , " ")
                para = para.strip()
                #print(para)

                title = j_import['title']
                title = title.replace("	", "").replace("\n", ' ')
                title = title.replace("\r" , " ")
                title = title.strip()

                author = j_import['author']
                author = author.replace("	", "").replace("\n", ' ')
                author = author.replace("\r" , " ")
                author = author.strip()

                pubtime_i = j_import['pubtime']
                pubtime_i = pubtime_i[:19]

                pubtime = datetime(*time.strptime(pubtime_i, "%Y-%m-%dT%H:%M:%S")[:6])

                new_row = {"title" : title , "para" : para , "author": author ,
                           "pubtime" : pubtime }
                data = data.append(new_row, ignore_index=True)
                f.close()
            else:
                pass

        except KeyError:
            f.close()
            pass

    except json.decoder.JSONDecodeError:
        f.close()
        pass

#data = data.set_index('title')
data

,author,para,pubtime,title
0,Artnet News,Art Industry News is a daily digest of the mos...,2019-03-20 13:16:19,Art Industry News: Obama Portraitist Kehinde W...
1,Caroline Goldstein,In another installment of our series investiga...,2019-11-08 17:18:05,"See the Poetic, Heartbreaking Final Paintings ..."
2,Brian Boucher,"As museum acquisitions grow, and amid a balloo...",2016-08-12 15:21:58,"From Auschwitz Drawings to Teddy Bears, ‘The K..."
3,Vivienne Chow,"On June 30, the eve of the 23rd anniversary of...",2020-07-16 12:59:43,Beijing Is Clamping Down on ‘Subversive’ Activ...
4,Taylor Dafoe,"A year ago, artist Allison Zuckerman was 26, f...",2017-12-05 14:14:32,Behind Artist Allison Zuckerman’s Rapid Rise F...
...,...,...,...,...
95,Kenny Schachter,"New York, New York Back in New York, I attende...",2016-09-23 20:00:22,Kenny Schachter’s Dealer Diary: Of Art and Car...
96,Brian Boucher,In a 1964 conversation with New Yorker scribe ...,2017-10-03 05:00:39,Has Duchamp’s Final Work Harbored a Secret for...
97,Artnet News,"This week is Koons Week New York, with Split-R...",2014-06-24 16:06:09,The Wisdom of Jeff Koons in 6 Easy Quotes
98,Eileen Kinsella Tim Schneider,"Just over a decade ago, the high-flying art ma...",2019-09-25 06:00:15,Is the Art Market Headed for Another Recession...


In [14]:
lemmatizer = nlp.get_pipe("lemmatizer")
print(lemmatizer.mode)  # 'rule'

from spacy.kb import KnowledgeBase
vocab = nlp.vocab
kb = KnowledgeBase(vocab=vocab, entity_vector_length=64)

orgs = []
for article in data['para'][:1]: 
    doc = nlp(article)
    for e in doc.ents:
        #ents = [(e.text, e.label_, e.kb_id_) 
        if e.label_ == 'ORG':
            orgs.append(e.text)
            

    
print(orgs)   
    
    
    
    #displacy.serve(doc, style="ent")
    
    
    #for ent in doc.ents:
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
    #for token in doc:
        #lemm = token.lemma_()
        #print(lemm)
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

rule


TypeError: add_entity() takes exactly 3 positional arguments (1 given)